### LLMChain

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain import PromptTemplate, OpenAI, LLMChain

prompt_template = "what is a word to replace the follwoing: {word}?"

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

llm_chain = LLMChain(
llm=llm,
prompt=PromptTemplate.from_template(prompt_template)
)

/Users/mirmustafaali/anaconda3/envs/activeloop/lib/python3.11/site-packages/langchain/llms/openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/mirmustafaali/anaconda3/envs/activeloop/lib/python3.11/site-packages/langchain/llms/openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [5]:
llm_chain("artificial")

{'word': 'artificial', 'text': 'synthetic'}

In [13]:
input_list = [
    {"word":"artificial"},
    {"word": "intelligence"},
    {"word":"robot"}
]

llm_chain(input_list[0])

{'word': 'artificial', 'text': 'synthetic'}

In [23]:
llm_chain.generate([input_list[0]])

LLMResult(generations=[[Generation(text='synthetic', generation_info=None)]], llm_output={'token_usage': <OpenAIObject at 0x140a4f7d0> JSON: {
  "prompt_tokens": 20,
  "completion_tokens": 2,
  "total_tokens": 22
}, 'model_name': 'gpt-3.5-turbo'}, run=RunInfo(run_id=UUID('ba123767-54ed-4736-a0ff-092f643eebad')))

In [28]:
prompt_template = """Looking at the context of '{context}'. \
What is an appropriate word to replace the following: {word}"""
llm_chain = LLMChain(llm=llm, prompt=PromptTemplate(template=prompt_template, input_variables=["word", "context"]))
llm_chain.predict(word="fan", context="object")

'air conditioner'

In [37]:
llm_chain = LLMChain.from_string(llm=llm, template=prompt_template)
llm_chain.predict(word="fan", context="human")

'supporter'

### Conversational Chain

In [43]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import CommaSeparatedListOutputParser


output_parser = CommaSeparatedListOutputParser()
conversation = ConversationChain(
llm = llm,
memory = ConversationBufferMemory())


conversation.predict(input="""List all possible words as substitute for 'artifical' as comma separated.""")




'synthetic, simulated, man-made, manufactured, fake, imitation'

In [41]:
conversation.predict(input="And the next 4?")

'replica, imitation, faux, pseudo'

### Sequential Chain
```
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chain=[chain_one, chain_two])
```

### Debug

In [45]:
template = """List all possible words as substitute for 'artificial' as comma separated.
Current conversation:
{history}

{input}"""


conversation = ConversationChain(
llm=llm,
    prompt=PromptTemplate(template=template,
                         input_variables=["history", "input"], output_parser=output_parser),
    memory = ConversationBufferMemory(),
    verbose=True )
conversation.predict(input="")



> Entering new  chain...
Prompt after formatting:
List all possible words as substitute for 'artificial' as comma separated.
Current conversation:




> Finished chain.


'synthetic, man-made, fake, imitation, simulated, faux, counterfeit, ersatz, fabricated, manufactured'

## Custom Chain

In [46]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain


from typing import Dict, List


class ConcatenatedChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain
        
        
    @property
    def input_keys(self) -> List[str]:
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)
    
    
    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']
    
    
    def _call(self, inputs:Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

In [48]:
prompt_1 = PromptTemplate(
input_variables = ["word"],
template = "What is the meaning of the following word '{word}'?")

chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2= PromptTemplate(
input_variables=["word"],
template = "What is a word to replace the following: {word}?")

chain_2 = LLMChain(llm=llm, prompt=prompt_2)


concat_chain = ConcatenatedChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
print(f"concatenated output: \n {concat_output}")

concatenated output: 
 Artificial means made or produced by human beings rather than occurring naturally. It can also refer to something that is not genuine or real.Synthetic
